In [1]:
# notebooks/03_data_merging.ipynb

import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler
import sys
import os

# Add src to path
sys.path.append(os.path.abspath('..'))
from src.utils import add_technical_indicators

# 1. SETUP & LOAD
print("Loading data...")
df_prices = pd.read_csv('../data/raw/hsi_price_history.csv')
df_sentiment = pd.read_csv('../data/processed/daily_sentiment.csv')

# --- FIX START: CLEAN UP YFINANCE DATA STRUCTURE ---
# yfinance often saves a 'Ticker' column or creates a MultiIndex (Price, Ticker)
# We flatten it and remove non-numeric columns immediately.

# If 'Ticker' exists as a column, drop it
if 'Ticker' in df_prices.columns:
    print("Dropping 'Ticker' column...")
    df_prices = df_prices.drop(columns=['Ticker'])

# If the dataframe has a MultiIndex columns (Level 0: Price, Level 1: Ticker), flatten it
if isinstance(df_prices.columns, pd.MultiIndex):
    print("Flattening MultiIndex columns...")
    df_prices.columns = df_prices.columns.get_level_values(0)

# Sometimes the first few rows are header metadata garbage, ensure 'Date' is real
# (This step depends on how exactly read_csv loaded it, but usually standard read_csv is fine)

# --- FIX END ---

# Convert dates
df_prices['Date'] = pd.to_datetime(df_prices['Date']).dt.tz_localize(None)
df_sentiment['Date'] = pd.to_datetime(df_sentiment['Date'])

# 2. MERGE
df_merged = pd.merge(df_prices, df_sentiment, on='Date', how='left')
df_merged['sentiment_score'] = df_merged['sentiment_score'].fillna(0)

# 3. ADD INDICATORS
print("Adding technical indicators...")
# Double check types before passing
# Force numeric on key columns
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in numeric_cols:
    if col in df_merged.columns:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

df_processed = add_technical_indicators(df_merged)

# Drop NaNs
df_processed.dropna(inplace=True)

# 4. SCALING
# Ensure we only scale numeric columns
valid_feature_cols = [col for col in df_processed.columns 
                     if col not in ['Date', 'Headline'] and 
                     pd.api.types.is_numeric_dtype(df_processed[col])]

print(f"Features to scale: {valid_feature_cols}")

scaler_all = MinMaxScaler()
df_processed[valid_feature_cols] = scaler_all.fit_transform(df_processed[valid_feature_cols])

target_scaler = MinMaxScaler()
target_scaler.fit(df_merged[['Close']].values)

# 5. SAVE ARTIFACTS
os.makedirs('../data/processed/scalers', exist_ok=True)
joblib.dump(scaler_all, '../data/processed/scalers/feature_scaler.pkl')
joblib.dump(target_scaler, '../data/processed/scalers/target_scaler.pkl')
df_processed.to_csv('../data/processed/training_data.csv', index=False)

print(f"Success! Saved {len(df_processed)} rows.")



Loading data...
Adding technical indicators...
Features to scale: ['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment_score', 'MA_5', 'MA_10', 'MA_20', 'MA_50', 'RSI', 'Price_Change', 'Volume_Change', 'Volatility']
Success! Saved 2658 rows.
